In [1]:
from brian2 import *
import numpy as np
from itertools import groupby
from operator import itemgetter
import pickle

In [2]:
# network param.
M = 25*5
L = 4
N = M*L;
N_E = N
N_readout = 400
N_extra_readout = 2

n_trials = 30
s = [0.25, 0.25 + 0.5]
s1 = s[0]
s2 = s[1]

save_to = './data/Periodic_population'

# neuron param.
tau_mem = 20*ms
thresh = 20 *mV
V_reset = 10*mV
V_rest = 0*mV

tau_rp = 2*ms

eqs = '''
dv/dt = -(v-V_rest)/tau_mem : volt (unless refractory)
'''

D = 1.5 *ms # Synaptic delay

# synapse param.
J_E = 0.2*mV
J_EE = 10*J_E
J_I = J_EE
#nu_thr = thresh / (J_E * tau_mem)

theta_0 = np.linspace(0,1,N + 1)
theta_0 = theta_0[:-1] #remove last element (circular stimulus variable)

# preferred locations of readout layer neurons
readout_theta_0 = np.linspace(0,1,N_readout + 1)
readout_theta_0 = readout_theta_0[:-1]

max_input = 5000* Hz #Heuristically set
non_specific = 0.85
specific = 0.15
#controlls width of input modulation to input layer
w = 0.3 
# Spatial frequency (1/(R*spatial period)) of input layer neurons
f_1 = 1
f_2 = 2
f_3 = 3
f_4 = 4

theta_0_module_1 = np.linspace(0,1/f_1, M + 1)
theta_0_module_1 = theta_0_module_1[:-1] #remove last element (circular stimulus variable)

theta_0_module_2 = np.linspace(0,1/f_2, M + 1)
theta_0_module_2 = theta_0_module_2[:-1] #remove last element (circular stimulus variable)

theta_0_module_3 = np.linspace(0,1/f_3, M + 1)
theta_0_module_3 = theta_0_module_3[:-1] #remove last element (circular stimulus variable)

theta_0_module_4 = np.linspace(0,1/f_4, M + 1)
theta_0_module_4 = theta_0_module_4[:-1] #remove last element (circular stimulus variable)

# Poisson input rates to the input layer
#0.82*5400
# Baseline input is 0.85*5000 = 4250 just enough to make neurons spike a few times, rest 0.15*5000 = 750 is stimulus modulation
input_rates_module_1 = lambda theta: (specific*exp(1/w*(cos(2*pi*f_1*(theta-theta_0_module_1)) - 1)) + non_specific)*max_input
input_rates_module_2 = lambda theta: (specific*exp(1/w*(cos(2*pi*f_2*(theta-theta_0_module_2)) - 1)) + non_specific )*max_input
input_rates_module_3 = lambda theta: (specific*exp(1/w*(cos(2*pi*f_3*(theta-theta_0_module_3)) - 1)) + non_specific )*max_input
input_rates_module_4 = lambda theta: (specific*exp(1/w*(cos(2*pi*f_4*(theta-theta_0_module_4)) - 1)) + non_specific )*max_input

In [3]:
#activity input to input layer

defaultclock.dt = 0.1 * ms
TC_module_1 = [list(input_rates_module_1(min(1,max(s[i],0)))) for i in range(len(s[:]))]
TC_module_2 = [list(input_rates_module_2(min(1,max(s[i],0)))) for i in range(len(s[:]))]
TC_module_3 = [list(input_rates_module_3(min(1,max(s[i],0)))) for i in range(len(s[:]))]
TC_module_4 = [list(input_rates_module_4(min(1,max(s[i],0)))) for i in range(len(s[:]))]


rate_module_1_vec = TimedArray(TC_module_1, dt=500*ms)
rate_module_2_vec = TimedArray(TC_module_2, dt=500*ms)
rate_module_3_vec = TimedArray(TC_module_3, dt=500*ms)
rate_module_4_vec = TimedArray(TC_module_4, dt=500*ms)

In [6]:
for k in range(0,n_trials):
    # Get input to the first layer
    start_scope()

    defaultclock.dt = 0.1 * ms
    P_1 = PoissonGroup(M, rates='rate_module_1_vec(t, i)')
    P_2 = PoissonGroup(M, rates='rate_module_2_vec(t, i)')
    P_3 = PoissonGroup(M, rates='rate_module_3_vec(t, i)')
    P_4 = PoissonGroup(M, rates='rate_module_4_vec(t, i)')

    MP_1 = SpikeMonitor(P_1)
    MP_2 = SpikeMonitor(P_2)
    MP_3 = SpikeMonitor(P_3)
    MP_4 = SpikeMonitor(P_4)

    run(1000*ms)
    # And keep a copy of those spikes
    spikes_i_M_1 = MP_1.i
    spikes_t_M_1 = MP_1.t

    spikes_i_M_2 = MP_2.i
    spikes_t_M_2 = MP_2.t

    spikes_i_M_3 = MP_3.i
    spikes_t_M_3 = MP_3.t

    spikes_i_M_4 = MP_4.i
    spikes_t_M_4 = MP_4.t
    
    # Get baseline input activity to readout layer
    start_scope()
    defaultclock.dt = 0.1 * ms
    P = PoissonGroup(N_readout, rates=non_specific*max_input)
    MP = SpikeMonitor(P)
    run(1000*ms)
    # And keep a copy of those spikes
    spikes_i_readout_base = MP.i
    spikes_t_readout_base = MP.t
    
    
    ### Now we run the spiking network given the sampled inputs
    start_scope()
    defaultclock.dt = 0.1 * ms
    module_1 = NeuronGroup(M, eqs,
                              threshold="v > thresh",
                              reset="v = V_reset",
                              refractory=tau_rp,
                              method="exact"
                         )

    module_2 = NeuronGroup(M, eqs,
                              threshold="v > thresh",
                              reset="v = V_reset",
                              refractory=tau_rp,
                              method="exact"
                         )

    module_3 = NeuronGroup(M, eqs,
                              threshold="v > thresh",
                              reset="v = V_reset",
                              refractory=tau_rp,
                              method="exact"
                         )

    module_4 = NeuronGroup(M, eqs,
                              threshold="v > thresh",
                              reset="v = V_reset",
                              refractory=tau_rp,
                              method="exact"
                         )

    readout_neurons = NeuronGroup(N_readout, eqs,
                              threshold="v > thresh",
                              reset="v = V_reset",
                              refractory=tau_rp,
                              method="exact"
                         )
    

    module_1.v = 10*(1 + rand(M))*mV # initial value
    module_2.v = 10*(1 + rand(M))*mV # initial value
    module_3.v = 10*(1 + rand(M))*mV # initial value
    module_4.v = 10*(1 + rand(M))*mV # initial value

    readout_neurons.v = 10*(1 + rand(N_readout))*mV # initial value

    #Synapses input activity -> input layer
    SGG_1 = SpikeGeneratorGroup(M, spikes_i_M_1, spikes_t_M_1)
    external_syn_M_1 = Synapses(SGG_1, target=module_1, on_pre='v += J_E')
    external_syn_M_1.connect(j='i') #connect 1-to-1

    SGG_2 = SpikeGeneratorGroup(M, spikes_i_M_2, spikes_t_M_2)
    external_syn_M_2 = Synapses(SGG_2, target=module_2, on_pre='v += J_E')
    external_syn_M_2.connect(j='i') #connect 1-to-1

    SGG_3 = SpikeGeneratorGroup(M, spikes_i_M_3, spikes_t_M_3)
    external_syn_M_3 = Synapses(SGG_3, target=module_3, on_pre='v += J_E')
    external_syn_M_3.connect(j='i') #connect 1-to-1

    SGG_4 = SpikeGeneratorGroup(M, spikes_i_M_4, spikes_t_M_4)
    external_syn_M_4 = Synapses(SGG_4, target=module_4, on_pre='v += J_E')
    external_syn_M_4.connect(j='i') #connect 1-to-1

    #Synapses input layer -> readout layer (distance dependent EPSPs - in stimulus space)
    read_out_syn_M_1 = Synapses(module_1, readout_neurons, 'epsp : volt', on_pre='v += epsp', delay = D)
    read_out_syn_M_2 = Synapses(module_2, readout_neurons, 'epsp : volt', on_pre='v += epsp', delay = D)
    read_out_syn_M_3 = Synapses(module_3, readout_neurons, 'epsp : volt', on_pre='v += epsp', delay = D)
    read_out_syn_M_4 = Synapses(module_4, readout_neurons, 'epsp : volt', on_pre='v += epsp', delay = D)

    read_out_syn_M_1.connect()
    read_out_syn_M_2.connect()
    read_out_syn_M_3.connect()
    read_out_syn_M_4.connect()

    FWHM = 1/N_readout
    sigma = (FWHM/2) / sqrt(2*log(2));
    w_readout = (pi*FWHM)**2/(2*log(2))
    read_out_syn_M_1.epsp = 'exp(1/(w_readout)*(cos(2*pi*f_1*(1/M*i/f_1 - 1/N_readout*j))-1))*J_EE'
    read_out_syn_M_2.epsp = 'exp(1/(w_readout)*(cos(2*pi*f_2*(1/M*i/f_2 - 1/N_readout*j))-1))*J_EE'
    read_out_syn_M_3.epsp = 'exp(1/(w_readout)*(cos(2*pi*f_3*(1/M*i/f_3 - 1/N_readout*j))-1))*J_EE'
    read_out_syn_M_4.epsp = 'exp(1/(w_readout)*(cos(2*pi*f_4*(1/M*i/f_4 - 1/N_readout*j))-1))*J_EE'

    #Synapses baseline input to readout layer
    SGG_readout = SpikeGeneratorGroup(N_readout, spikes_i_readout_base, spikes_t_readout_base)
    external_syn_readout = Synapses(SGG_readout, target=readout_neurons, on_pre='v += J_E')
    external_syn_readout.connect(j='i') #connect 1-to-1

    #Winner take all style inhibitory synapses between readout layer (distance dependent, "nearer" -> less inhibition)
    read_out_syn_inhib = Synapses(readout_neurons, readout_neurons, 'ipsp : volt', on_pre='v += ipsp', delay = D)
    read_out_syn_inhib.connect()
    read_out_syn_inhib.ipsp = '-abs(sin(pi*(1/N_readout*i - 1/N_readout*j)))*J_I'
    #read_out_syn_inhib.ipsp = '-abs(sin(pi*((s1*(1-i) + s2*i) - (s1*(1-j) + s2*j))))*J_I'
    
    
    spike_monitor_M_1 = SpikeMonitor(module_1)
    spike_monitor_M_2 = SpikeMonitor(module_2)
    spike_monitor_M_3 = SpikeMonitor(module_3)
    spike_monitor_M_4 = SpikeMonitor(module_4)

    spike_monitor_readout = SpikeMonitor(readout_neurons)

    run(1000*ms)
        
    data_1 = spike_monitor_M_1.get_states(['t','i','count'])
    data_2 = spike_monitor_M_2.get_states(['t','i','count'])
    data_3 = spike_monitor_M_3.get_states(['t','i','count'])
    data_4 = spike_monitor_M_4.get_states(['t','i','count'])
    
    data_readout = spike_monitor_readout.get_states(['t','i','count'])

    with open(save_to + '/data_TC_layer_run_idx_{}.pickle'.format(k), 'wb') as save_file:
        pickle.dump(data_1, save_file)
        pickle.dump(data_2, save_file)
        pickle.dump(data_3, save_file)
        pickle.dump(data_4, save_file)

    with open(save_to + '/data_readout_layer_run_idx_{}.pickle'.format(k), 'wb') as save_file:
        pickle.dump(data_readout, save_file)
        
        